# Introduction

* Name: / Javier / Χαβιέρ 
* Digital A.I.R
* Python-fan-person

# Pickles

In [1]:
class Account:
    def __init__(self, name):
        self.amount = 0
        self.name = name

    def add(self, amount):
        self.amount += amount

my_account = Account("Javier")
my_account.add(42)
my_account.amount

42

In [2]:
import pickle

account_pickle = pickle.dumps(my_account)
account_pickle

b'\x80\x03c__main__\nAccount\nq\x00)\x81q\x01}q\x02(X\x04\x00\x00\x00nameq\x03X\x06\x00\x00\x00Javierq\x04X\x06\x00\x00\x00amountq\x05K*ub.'

In [3]:
other_account = pickle.loads(account_pickle)
print(other_account.amount, other_account.name)


42 Javier


In [4]:
(id(my_account),id(other_account))

(4452866536, 4452918160)

# ZODB

In [5]:
import persistent

class Account(persistent.Persistent):
    def __init__(self):
        self.__amount = 0
    @property
    def amount(self):
        return self.__amount

    def add(self, amount):
        self.__amount += amount

In [6]:
import ZODB
import ZODB.FileStorage        
        

storage = ZODB.FileStorage.FileStorage('bank.fs')
db = ZODB.DB(storage)
connection = db.open()
root = connection.root

myaccount = Account()
myaccount.add(50)
root.myaccount = myaccount
print(root)

storage.close()
db.close()




<root: accounts myaccount>


In [7]:
storage = ZODB.FileStorage.FileStorage('bank.fs')
db = ZODB.DB(storage)
connection = db.open()
root = connection.root

print(root)

<root: accounts>


In [8]:
import transaction

myaccount = Account()
myaccount.add(50)
print(myaccount.amount)
root.myaccount = myaccount
transaction.commit()

storage.close()
db.close()

50


In [9]:
storage = ZODB.FileStorage.FileStorage('bank.fs')
db = ZODB.DB(storage)
connection = db.open()
root = connection.root

print(root.myaccount.amount)


storage.close()
db.close()

50


# Indexes

In [10]:
class DemoZODB:
    def __init__(self, filename):
        self.storage = ZODB.FileStorage.FileStorage(filename)
        self.db = ZODB.DB(self.storage)
        self.connection = self.db.open()
        self.root = self.connection.root
    def __enter__(self):
        return self.root
    def __exit__(self, type, value, traceback):
        transaction.commit()
        self.db.close()
        self.storage.close()

In [11]:
from BTrees.OOBTree import OOBTree

with DemoZODB('bank.fs') as root:
    root.accounts = OOBTree()
    root.accounts["Javier"] = root.myaccount
    del root.myaccount

with DemoZODB('bank.fs') as root:
    print(root.accounts["Javier"].amount)
    

50


## MOAR Indexes

`repoze.catalog` Only for python 2.x

```
from repoze.catalog.indexes.field import CatalogFieldIndex
​
  File "/Users/alff/Projects/pybkk/.env/lib/python3.5/site-packages/repoze/catalog/indexes/field.py", line 183
    raise RuntimeError, 'n-best used without limit'
                      ^
SyntaxError: invalid syntax
```

# ZeroDB

In [12]:
from zerodb.models import Model, Field

class Account(Model):
    owner = Field()
    amount = Field()

    def __init__(self, owner):
        self.amount = 0
        self.owner = owner
    def substract(self, amount):
        self.amount -= amount
    def add(self, amount):
        self.amount += amount
        
my_account = Account("Javier")
my_account.add(42)
dog_account = Account("Dog")

In [13]:
username = "javier"
password = "javier"

import transaction
import zerodb


db = zerodb.DB(("localhost", 8001), username=username, password=password)
db.add(my_account)
transaction.get().note("Adding my account")
transaction.commit()

db.add(dog_account)
transaction.get().note("Adding dog account")
transaction.commit()
db.disconnect()

In [14]:
db = zerodb.DB(("localhost", 8001), username=username, password=password)
print([x.owner for x in db[Account].all()])
db.disconnect()


['Javier', 'Dog']


In [15]:
def list_dog():
    db = zerodb.DB(("localhost", 8001), username="dog", password="dog")
    print([x.owner for x in db[Account].all()])
    db.disconnect()
list_dog()

[]


In [16]:
db = zerodb.DB(("localhost", 8001), username="dog", password="dog")
db.add(Account("Someone"))
transaction.commit()
db.disconnect()

In [17]:
list_dog()

['Someone']


## Indexing

In [18]:
from zerodb.query import Gt

db = zerodb.DB(("localhost", 8001), username=username, password=password)
print([x.owner for x in db[Account].query(Gt("amount", 4))])
db.disconnect()



['Javier']
